## Table of Contents

- [Introduction](#introduction)

- [Links of Interest](#links)

- [Loading Data](#paths)
    - [Setting input and output paths](#paths)
    - [Reading in a dataset of text descriptions](#read_this_data_)
    - [Reading in a dataset of groups or categories](#read_other_data)
    - [Relating the datasets to one another](#relating)
    - [Filtering the datasets](#filtering)
    
- [Language Models](#load_models)
    - [Loading models](#load_models)
    - [BERT and BioBERT]()
    - [Word2Vec and Doc2Vec]()
    
- [NLP Choices](#part8)
    - [Creating vocabularies](#vocab)
    - [Preprocessing descriptions](#preprocessing)
    - [POS Tagging](#pos_tagging)
    - [Annotating descriptions](#annotation)
    
- [Building a Distance Matrix](#matrix)
    - [Defining a list of methods to use](#methods)
    - [Running each method](#run)
    - [Adding additional information](#merging)
    - [Combining methods with ensemble models](#ensemble)
- [Analysis]()
    - [Comparing distributions of distance values between methods](#ks)
    - [Comparing the within-group distance values across gene groups and methods](#within)
    - [Comparing the AUC for predicting shared pathways, gene groups, or interactions between methods](#auc)
    - [Comparing querying for similar genes using distance matrices for each method](#y)
    - [Comparing the AUC for predicting the specific pathway or group of a gene](#mean)
    - [Generating a table of resulting metrics for each method](#output)
    

<a id="introduction"></a>
### Introduction: Text Mining Analysis of Phenotype Descriptions in Plants
The purpose of this notebook is to evaluate what can be learned from a natural language processing approach to analyzing free-text descriptions of phenotype descriptions of plants. The approach is to generate pairwise distances matrices between a set of plant phenotype descriptions across different species, sourced from academic papers and online model organism databases. These pairwise distance matrices can be constructed using any vectorization method that can be applied to natural language. In this notebook, we specifically evaluate the use of n-gram and bag-of-words techniques, word and document embedding using Word2Vec and Doc2Vec, context-dependent word-embeddings using BERT and BioBERT, and ontology term annotations with automated annotation tools such as NOBLE Coder.

Loading, manipulation, and filtering of the dataset of phenotype descriptions associated with genes across different plant species is largely handled through a Python package created for this purpose called OATS (Ontology Annotation and Text Similarity) which is available [here](https://github.com/irbraun/oats). Preprocessing of the descriptions, mapping the dataset to additional resources such as protein-protein interaction databases and biochemical pathway databases are handled in this notebook using that package as well. In the evaluation of each of these natural language processing approaches to analyzing this dataset of descriptions, we compare performance against a dataset generated through manual annotation of a similar dataset in Oellrich Walls et al. (2015) and against manual annotations with experimentally determined terms from the Gene Ontology (PO) and the Plant Ontology (PO).

<a id="links"></a>
### Relevant Links of Interest:
- Paper describing comparison of NLP and ontology annotation approaches to curation: [Braun, Lawrence-Dill (2019)](https://doi.org/10.3389/fpls.2019.01629)
- Paper describing results of manual phenotype description curation: [Oellrich, Walls et al. (2015](https://plantmethods.biomedcentral.com/articles/10.1186/s13007-015-0053-y)
- Plant databases with phenotype description text data available: [TAIR](https://www.arabidopsis.org/), [SGN](https://solgenomics.net/), [MaizeGDB](https://www.maizegdb.org/)
- Python package for working with phenotype descriptions: [OATS](https://github.com/irbraun/oats)
- Python package used for general NLP functions: [NLTK](https://www.nltk.org/)
- Python package used for working with biological ontologies: [Pronto](https://pronto.readthedocs.io/en/latest/)
- Python package for loading pretrained BERT models: [PyTorch Pretrained BERT](https://pypi.org/project/pytorch-pretrained-bert/)
- For BERT Models pretrained on PubMed and PMC: [BioBERT Paper](https://arxiv.org/abs/1901.08746), [BioBERT Models](https://github.com/naver/biobert-pretrained)

In [1]:
import datetime
import nltk
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import pandas as pd
import numpy as np
import time
import math
import sys
import gensim
import os
import warnings
import torch
import itertools
import multiprocessing as mp
from collections import Counter, defaultdict
from inspect import signature
from scipy.stats import ks_2samp
from sklearn.metrics import precision_recall_curve, f1_score, auc
from sklearn.model_selection import train_test_split, KFold
from scipy import spatial, stats
from nltk.corpus import brown
from nltk.tokenize import word_tokenize
from sklearn.neighbors import KNeighborsClassifier
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM
from gensim.parsing.preprocessing import strip_non_alphanum, stem_text, preprocess_string, remove_stopwords
from gensim.utils import simple_preprocess

sys.path.append("../../oats")
from oats.utils.utils import save_to_pickle, load_from_pickle, merge_list_dicts, flatten, to_hms
from oats.datasets.dataset import Dataset
from oats.datasets.groupings import Groupings
from oats.annotation.ontology import Ontology
from oats.datasets.string import String
from oats.datasets.edges import Edges
from oats.annotation.annotation import annotate_using_noble_coder
from oats.graphs import pairwise as pw
from oats.graphs.indexed import IndexedGraph
from oats.graphs.weighting import train_logistic_regression_model, apply_logistic_regression_model
from oats.graphs.weighting import train_random_forest_model, apply_random_forest_model
from oats.nlp.vocabulary import get_overrepresented_tokens, build_vocabulary_from_tokens
from oats.nlp.vocabulary import collapse_vocabulary_by_distance
from oats.utils.utils import function_wrapper_with_duration
from oats.nlp.preprocess import concatenate_with_bar_delim

from _utils import Method

mpl.rcParams["figure.dpi"] = 400
warnings.simplefilter('ignore')
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
nltk.download('punkt', quiet=True)
nltk.download('brown', quiet=True)
nltk.download('averaged_perceptron_tagger', quiet=True)

True

<a id="paths"></a>
### Setting up the output table and input and output filepaths
This section defines some constants which are used for creating a uniquely named directory to contain all the outputs from running this instance of this notebook. The naming scheme is based on the time that the notebook is run. The other constants are used for specifying information in the output table about what the topic was for this notebook when it was run, such as looking at KEGG biochemical pathways or STRING protein-protein interaction data some other type of gene function grouping or hierarchy. These values are arbitrary and are just for keeping better notes about what the output of the notebook corresponds to. All the input and output file paths for loading datasets or models are also contained within this cell, so that if anything is moved the directories and file names should only have to be changed at this point and nowhere else further into the notebook. If additional files are added to the notebook cells they should be put here as well.

In [2]:
# The summarizing output dictionary has the shape TABLE[method][metric] --> value.
TOPIC = "Biochemical Pathways"
DATA = "Filtered"
TABLE = defaultdict(dict)
OUTPUT_DIR = os.path.join("../outputs",datetime.datetime.now().strftime('%m_%d_%Y_h%Hm%Ms%S'))
os.mkdir(OUTPUT_DIR)

In [3]:
dataset_filename = "../data/pickles/text_plus_annotations_dataset.pickle"        # The full dataset pickle.
groupings_filename = "../data/pickles/lloyd_subsets.pickle"                      # The groupings pickle.
background_corpus_filename = "../data/corpus_related_files/untagged_text_corpora/background.txt"       # Text file with background content.
phenotypes_corpus_filename = "../data/corpus_related_files/untagged_text_corpora/phenotypes_small.txt" # Text file with specific content.
doc2vec_pubmed_filename = "../gensim/pubmed_dbow/doc2vec_2.bin"                  # File holding saved Doc2Vec model.
doc2vec_wikipedia_filename = "../gensim/enwiki_dbow/doc2vec.bin"                 # File holding saved Doc2Vec model.
word2vec_model_filename = "../gensim/wiki_sg/word2vec.bin"                       # File holding saved Word2Vec model.
ontology_filename = "../ontologies/mo.obo"                                       # Ontology file in OBO format.
noblecoder_jarfile_path = "../lib/NobleCoder-1.0.jar"                            # Jar for NOBLE Coder tool.
biobert_pmc_path = "../gensim/biobert_v1.0_pmc/pytorch_model"                    # Path for PyTorch BioBERT model.
biobert_pubmed_path = "../gensim/biobert_v1.0_pubmed/pytorch_model"              # Path for PyTorch BioBERT model.
biobert_pubmed_pmc_path = "../gensim/biobert_v1.0_pubmed_pmc/pytorch_model"      # Path for PyTorch BioBERT model.

<a id="read_this_data"></a>
### Reading in the dataset of genes and their associated phenotype descriptions and annotations

In [4]:
dataset = load_from_pickle(dataset_filename)
dataset.describe()
dataset.filter_by_species("ath")
dataset.filter_has_description()
#dataset.filter_has_annotation()
dataset.describe()
#dataset.filter_has_annotation("GO")
#dataset.filter_has_annotation("PO")
dataset.describe()
dataset.to_pandas().head(10)

Number of rows in the dataframe: 30169
Number of unique IDs:            30169
Number of unique descriptions:   4566
Number of unique gene name sets: 30169
Number of species represented:   6
Number of rows in the dataframe: 6030
Number of unique IDs:            6030
Number of unique descriptions:   3611
Number of unique gene name sets: 6030
Number of species represented:   1
Number of rows in the dataframe: 6030
Number of unique IDs:            6030
Number of unique descriptions:   3611
Number of unique gene name sets: 6030
Number of species represented:   1


,id,species,gene_names,description,term_ids
0,0,ath,At3g49600|UBP26|AT3G49600|SUP32|ATUBP26|ubiqui...,50% defective seeds. Low penetrance of endospe...,GO:0005730|GO:0048316|PO:0000013|PO:0000037|PO...
1,1,ath,AT5G25620|YUC6|AtYUC6|YUCCA6|T14C9.160|T14C9_160,"epinastic cotyledons, long hypocotyls, curled ...",GO:0009851|PO:0009031|PO:0000293|PO:0007611
2,2,ath,AT5G25220|KNAT3|KNOTTED1-like homeobox gene 3|...,No visible root phenotype. Normal embryo sac d...,GO:0005829|GO:0071345|GO:0009416|GO:0005515|GO...
3,3,ath,AT5G24860|FPF1|ATFPF1|FLOWERING PROMOTING FACT...,late flowering.,GO:0009739|GO:0010228|GO:0003674|GO:0009911|PO...
4,4,ath,AT5G24740|SHBY|SHRUBBY|T4C12.10,no visible developmental phenotypes were obser...,GO:0010082|GO:0010015|GO:1990064|PO:0000282|PO...
5,5,ath,AT5G24380|YSL2|ATYSL2|YELLOW STRIPE like 2|K16...,Morphology and development of mutant plants gr...,GO:0005886|GO:0010039|GO:0010043|PO:0005059|PO...
6,6,ath,AT5G24360|IRE1-1|ATIRE1-1|AtIRE1b|IRE1B|ARABID...,bZIP60 mRNA splicing is detectable in ire1b-1 ...,GO:0034263|GO:0005783|GO:0009816|GO:0000394|GO...
7,7,ath,AT5G24330|ATXR6|SDG34|ARABIDOPSIS TRITHORAX-RE...,reduced H3K27me1 in vivo; partial heterochroma...,GO:0006275|GO:0070734|GO:0005634|GO:0009901|GO...
8,8,ath,AT5G24300|SS1|ATSS1|STARCH SYNTHASE 1|MOP9.12|...,"Germination percentages, rates of growth,flowe...",GO:0009011|GO:0009570|GO:0009507|GO:0010021|PO...
9,9,ath,AT5G24290|MEB2|MEMBRANE OF ER BODY 2|MOP9.11|M...,The plants do not accumulate MEB2 protein. The...,GO:0005515|GO:0010168|GO:0005384|GO:0005381|PO...


<a id="read_other_data"></a>
### Reading in the dataset of groupings, pathways, or some other kind of categorization

In [5]:
groups = load_from_pickle(groupings_filename)
id_to_group_ids = groups.get_id_to_group_ids_dict(dataset.get_gene_dictionary())
group_id_to_ids = groups.get_group_id_to_ids_dict(dataset.get_gene_dictionary())
group_mapped_ids = [k for (k,v) in id_to_group_ids.items() if len(v)>0]
groups.describe()
groups.to_csv(os.path.join(OUTPUT_DIR,"groupings.csv"))
groups.to_pandas().head(10)

Number of groups present for each species
  ath: 42
Number of genes names mapped to any group for each species
  ath: 7620


,species,group_id,gene_names
0,ath,FSM,at1g01030|nga3|top1|ngatha
1,ath,EMB|FSM|OVP|SRF,at1g01040|sus1|dcl1|sin1|caf|abnormal suspensor
2,ath,CDR|LIT,at1g01060|lhy|late elongated hypocotyl
3,ath,IST|WAT,at1g01120|kcs1|3-ketoacyl-coa synthase defective
4,ath,OVP|SRF,at1g01280|cyp703a2|cytochrome p450
5,ath,EMB,at1g01370|cenh3|centromere-specific histone
6,ath,CHS,at1g01460|pipk11|phosphatidylinositol phosphat...
7,ath,NLS|GRS|IST,at1g01480|acs2|aminocyclopropane carboxylate s...
8,ath,LEF|FSM,at1g01510|an|angustifolia
9,ath,SRL|ROT|LEF|MSL|STT|RTH|TCM|TMP,at1g01550|bps1|bypass


<a id="relating"></a>
### Relating the dataset of genes to the dataset of groupings or categories
This section generates tables that indicate how the genes present in the dataset were mapped to the defined pathways or groups. This includes a summary table that indicates how many genes by species were succcessfully mapped to atleast one pathway or group, as well as a more detailed table describing how many genes from each species were mapped to each particular pathway or group.

In [6]:
# Generate a table describing how many of the genes input from each species map to atleast one group.
summary = defaultdict(dict)
species_dict = dataset.get_species_dictionary()
for species in dataset.get_species():
    summary[species]["input"] = len([x for x in dataset.get_ids() if species_dict[x]==species])
    summary[species]["mapped"] = len([x for x in group_mapped_ids if species_dict[x]==species])
table = pd.DataFrame(summary).transpose()
table.loc["total"]= table.sum()
table["fraction"] = table.apply(lambda row: "{:0.4f}".format(row["mapped"]/row["input"]), axis=1)
table = table.reset_index(inplace=False)
table = table.rename({"index":"species"}, axis="columns")
table.to_csv(os.path.join(OUTPUT_DIR,"mappings_summary.csv"), index=False)

# Generate a table describing how many genes from each species map to which particular group.
summary = defaultdict(dict)
for group_id,ids in group_id_to_ids.items():
    summary[group_id].update({species:len([x for x in ids if species_dict[x]==species]) for species in dataset.get_species()})
    summary[group_id]["total"] = len([x for x in ids])
table = pd.DataFrame(summary).transpose()
table = table.sort_values(by="total", ascending=False)
table = table.reset_index(inplace=False)
table = table.rename({"index":"pathway_id"}, axis="columns")
table["pathway_name"] = table["pathway_id"].map(groups.get_long_name)
table.loc["total"] = table.sum()
table.loc["total","pathway_id"] = "total"
table.loc["total","pathway_name"] = "total"
table = table[table.columns.tolist()[-1:] + table.columns.tolist()[:-1]]
table.to_csv(os.path.join(OUTPUT_DIR,"mappings_by_group.csv"), index=False)

<a id="filtering"></a>
### Option 1: Filtering the dataset based on presence in the curated Oellrich, Walls et al. (2015) dataset

In [ ]:
# Filter the dataset based on whether or not the genes were in the curated dataset.
# This is similar to filtering based on protein interaction data because the dataset is a list of edge values.
pppn_edgelist_path = "../data/supplemental_files_oellrich_walls/13007_2015_53_MOESM9_ESM.txt"
pppn_edgelist = Edges(dataset.get_name_to_id_dictionary(), pppn_edgelist_path)
dataset.filter_with_ids(pppn_edgelist.ids)
dataset.filter_random_k(100)
dataset.describe()

### Option 2: Filtering the dataset based on protein-protein interactions
This is done to only include genes (and the corresponding phenotype descriptions and annotations) which are useful for the current analysis. In this case we want to only retain genes that are mentioned atleast one time in the STRING database for a given species. If a gene is not mentioned at all in STRING, there is no information available for whether or not it interacts with any other proteins in the dataset so choose to not include it in the analysis. Only genes that have atleast one true positive are included because these are the only ones for which the missing information (negatives) is meaningful. This should be run instead of the subsequent cell, or the other way around, based on whether or not protein-protein interactions is the prediction goal for the current analysis.

In [ ]:
# Filter the dataset based on whether or not the genes were successfully mapped to an interaction.
# Reduce size of the dataset by removing genes not mentioned in the STRING.
naming_file = "../data/group_related_files/string/all_organisms.name_2_string.tsv"
interaction_files = [
    "../data/group_related_files/string/3702.protein.links.detailed.v11.0.txt", # Arabidopsis thaliana
    "../data/group_related_files/string/4577.protein.links.detailed.v11.0.txt", # maize
    "../data/group_related_files/string/4530.protein.links.detailed.v11.0.txt", # tomato 
    "../data/group_related_files/string/4081.protein.links.detailed.v11.0.txt", # medicago
    "../data/group_related_files/string/3880.protein.links.detailed.v11.0.txt", # rice 
    "../data/group_related_files/string/3847.protein.links.detailed.v11.0.txt", # soybean
]
genes = dataset.get_gene_dictionary()
string_data = String(genes, naming_file, *interaction_files)
dataset.filter_with_ids(string_data.ids)
dataset.describe()

### Option 3: Filtering the dataset based on membership in pathways or phenotype category
This is done to only include genes (and the corresponding phenotype descriptions and annotations) which are useful for the current analysis. In this case we want to only retain genes that are mapped to atleast one pathway in whatever the source of pathway membership we are using is (KEGG, Plant Metabolic Network, etc). This is because for these genes, it will be impossible to correctly predict their pathway membership, and we have no evidence that they belong or do not belong in certain pathways so they can not be identified as being true or false negatives in any case.

In [7]:
# Filter based on succcessful mappings to groups or pathways.
dataset.filter_with_ids(group_mapped_ids)
dataset.describe()
# Get the mappings in each direction again now that the dataset has been subset.
id_to_group_ids = groups.get_id_to_group_ids_dict(dataset.get_gene_dictionary())
group_id_to_ids = groups.get_group_id_to_ids_dict(dataset.get_gene_dictionary())

Number of rows in the dataframe: 2557
Number of unique IDs:            2557
Number of unique descriptions:   2167
Number of unique gene name sets: 2557
Number of species represented:   1


In [54]:
# Making an LDA figure from that to see if the topics match up, for Lloyd subsets.

descriptions = dataset.get_description_dictionary()
descriptions_simple_preprocessing = {i:" ".join(simple_preprocess(d)) for i,d in descriptions.items()}
tokens = list(set([w for w in flatten(word_tokenize(d) for d in descriptions_simple_preprocessing.values())]))
tokens_dict = {i:w for i,w in enumerate(tokens)}
graph = pw.pairwise_square_word2vec(word2vec_model, tokens_dict, "cosine")

# Get the objects needed to collapse the vocabulary and make a new vocabulary.
# Make sure that the tokens list is in the same order as the indices representing each word in the distance matrix.
# This is only trivial here because the IDs used are ordered integers 0 to n, but this might not always be the case.
distance_matrix = graph.array
tokens = [tokens_dict[graph.row_index_to_id[index]] for index in np.arange(distance_matrix.shape[0])]
threshold = 0.1
vocab, transform, untransform =  collapse_vocabulary_by_distance(tokens, distance_matrix, threshold)

In [67]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
from sklearn.decomposition import LatentDirichletAllocation as LDA


# The list of texts has to be created from the same descriptions used to create the reduced vocab (simple preprocess).
to_reduced_vocab = lambda t: " ".join([transform.get(w,"") for w in t.split(" ")])
texts = [to_reduced_vocab(d) for d in descriptions_simple_preprocessing.values()]
vectorizer = TfidfVectorizer(max_features=10000, stop_words="english", max_df=0.95, min_df=2, lowercase=False)
features = vectorizer.fit_transform(texts)
cls = NMF(n_components=40, random_state=0)
cls.fit(features)

# Function for retrieving the topic vectors for a list of text descriptions.
def get_topic_embeddings(texts, model, vectorizer):
    ngrams_vectors = vectorizer.transform(texts).toarray()
    topic_vectors = model.transform(ngrams_vectors)
    return(topic_vectors)
    
# Create the dataframe containing the average score assigned to each topic for the genes from each subset.
group_to_topic_vector = {}
for group_id,ids in group_id_to_ids.items():
    texts = [descriptions_simple_preprocessing[i] for i in ids]
    texts = [to_reduced_vocab(d) for d in texts]
    topic_vectors = get_topic_embeddings(texts, cls, vectorizer)
    mean_topic_vector = np.mean(topic_vectors, axis=0)
    group_to_topic_vector[group_id] = mean_topic_vector
    
df = pd.DataFrame(group_to_topic_vector)

# Changing the order of the Lloyd, Meinke phenotype subsets to match other figures for consistency.
filename = "../data/group_related_files/lloyd/lloyd_function_hierarchy_irb_cleaned.csv"
lmdf = pd.read_csv(filename)
df = df[lmdf["Subset Symbol"].values]

# Reordering so consistency with the curated subsets can be checked by looking at the diagonal.
df["idxmax"] = df.idxmax(axis = 1)
df["idxmax"] = df["idxmax"].apply(lambda x: df.columns.get_loc(x))
df = df.sort_values(by="idxmax")
df.drop(columns=["idxmax"], inplace=True)
df = df.reset_index(drop=False).rename({"index":"topic"},axis=1).reset_index(drop=False).rename({"index":"order"},axis=1)
df.to_csv("~/Desktop/hm.csv", index=False)
df

,order,topic,GAM,GEM,EMG,MGD,EMB,MSD,NHM,SRL,GER,NLS,PIG,GRS,ROT,LEF,IST,ARC,MSL,FSM,OVP,SRF,SSC,FLT,SEN,CDR,MTM,STT,RTH,TCM,CUL,PRA,CPR,WAT,TMP,LIT,MEC,MPH,NUT,HRM,CHS,MCH,PTH,OBI
0,0,2,0.206465,0.192433,0.142681,0.121135,0.006981,0.040561,0.000000,0.003730,0.007837,0.003482,0.001725,0.006544,0.007578,0.003693,0.001681,0.001899,0.003997,0.003619,0.003305,0.010721,0.000317,0.001600,0.001777,0.000247,0.002788,0.001106,0.006391,0.003825,0.001988,0.000941,0.002064,0.000519,0.003258,0.001169,0.000623,0.002805,0.001063,0.000982,0.001453,0.000086,0.000451,0.000000
1,1,0,0.004698,0.011868,0.021510,0.001570,0.058361,0.011149,0.000244,0.003034,0.000579,0.000541,0.003409,0.002428,0.001793,0.001888,0.000427,0.000178,0.005884,0.000774,0.001366,0.000784,0.000269,0.002231,0.000617,0.000106,0.000306,0.000111,0.000978,0.000765,0.004561,0.001064,0.000289,0.000132,0.001890,0.000220,0.000000,0.000074,0.000613,0.000517,0.000431,0.000370,0.000121,0.000007
2,2,21,0.008398,0.023307,0.028977,0.003173,0.038768,0.022700,0.000516,0.008301,0.001796,0.001207,0.007185,0.002461,0.001848,0.002379,0.003289,0.001270,0.003363,0.001301,0.000645,0.001692,0.000715,0.001363,0.001578,0.000286,0.003746,0.000908,0.002614,0.001799,0.003097,0.000763,0.000709,0.000347,0.001227,0.000603,0.000000,0.006052,0.000624,0.001341,0.000566,0.000690,0.000399,0.000043
3,3,9,0.001523,0.002149,0.019167,0.003671,0.035158,0.010354,0.003123,0.010431,0.006736,0.003205,0.005461,0.003117,0.002675,0.002888,0.003402,0.003523,0.003116,0.004961,0.002847,0.004114,0.001461,0.002066,0.002724,0.000291,0.001344,0.001765,0.001156,0.003401,0.001688,0.002910,0.000788,0.003205,0.004606,0.002541,0.000570,0.000855,0.003215,0.003368,0.001990,0.003492,0.002019,0.017424
4,4,8,0.003798,0.011468,0.018243,0.001391,0.057424,0.009802,0.000236,0.002956,0.000257,0.000801,0.002208,0.000958,0.001153,0.001055,0.002476,0.000172,0.000456,0.001224,0.000293,0.001013,0.000264,0.001256,0.000552,0.000112,0.000288,0.000148,0.000996,0.003264,0.008392,0.000741,0.000305,0.000153,0.001566,0.000247,0.000011,0.000084,0.001408,0.001087,0.001017,0.000282,0.001253,0.000000
5,5,6,0.011152,0.028868,0.039785,0.004846,0.087187,0.021400,0.000623,0.013663,0.003992,0.009933,0.009072,0.005224,0.006453,0.006993,0.005066,0.004211,0.003949,0.005423,0.001540,0.004602,0.006319,0.003832,0.004517,0.003934,0.000576,0.001303,0.006422,0.012197,0.005723,0.000821,0.000737,0.000409,0.002370,0.004514,0.006109,0.000258,0.001688,0.002375,0.001847,0.001538,0.001177,0.022140
6,6,33,0.009088,0.007088,0.008954,0.008810,0.004574,0.016767,0.262169,0.010049,0.004607,0.005871,0.008684,0.008692,0.005513,0.006923,0.005826,0.010912,0.012315,0.006254,0.005450,0.006750,0.005030,0.005617,0.008846,0.006278,0.006783,0.004649,0.003456,0.006584,0.011526,0.002679,0.005109,0.008421,0.008755,0.006405,0.005164,0.004237,0.004350,0.004317,0.003814,0.004692,0.004293,0.003737
7,7,5,0.013515,0.034613,0.044882,0.001020,0.002280,0.000256,0.000012,0.062232,0.000375,0.000853,0.052411,0.000483,0.000511,0.000630,0.000140,0.000431,0.000580,0.000757,0.000005,0.000649,0.013769,0.000322,0.005104,0.000016,0.000060,0.000849,0.001823,0.001531,0.000807,0.000539,0.001037,0.000035,0.001213,0.000596,0.002834,0.000003,0.000323,0.000201,0.000369,0.000468,0.000460,0.005364
8,8,18,0.001775,0.005562,0.005065,0.003878,0.006226,0.002865,0.005627,0.061448,0.009681,0.005633,0.024385,0.003478,0.003367,0.002834,0.001322,0.001868,0.002180,0.001689,0.000077,0.001238,0.000512,0.001773,0.003148,0.000726,0.001953,0.001177,0.000957,0.004714,0.004345,0.001234,0.001176,0.002048,0.004479,0.003657,0.008630,0.001450,0.005328,0.004031,0.001545,0.000566,0.001530,0.000297
9,9,26,0.003133,0.003484,0.001818,0.004276,0.002164,0.005713,0.000883,0.007208,0.059957,0.008508,0.005384,0.006608,0.006950,0.006083,0.005713,0.006012,0.004132,0.004155,0.006218,0.005454,0.010419,0.005614,0.013990,0.001030,0.004904,0.003196,0.000423,0.003834,0.003998,0.002381,0.002192,0.008992,0.016123,0.003653,0.003108,0.019695,0.003648,0.017404,0.011016,0.

<a id="load_models"></a>
### Loading trained and saved NLP neural network models

In [68]:
# Files and models related to the machine learning text embedding methods.
doc2vec_wiki_model = gensim.models.Doc2Vec.load(doc2vec_wikipedia_filename)
doc2vec_pubmed_model = gensim.models.Doc2Vec.load(doc2vec_pubmed_filename)
word2vec_model = gensim.models.Word2Vec.load(word2vec_model_filename)
#bert_tokenizer_base = BertTokenizer.from_pretrained('bert-base-uncased')
#bert_tokenizer_pmc = BertTokenizer.from_pretrained(biobert_pmc_path)
#bert_tokenizer_pubmed = BertTokenizer.from_pretrained(biobert_pubmed_path)
#bert_tokenizer_pubmed_pmc = BertTokenizer.from_pretrained(biobert_pubmed_pmc_path)
#bert_model_base = BertModel.from_pretrained('bert-base-uncased')
#bert_model_pmc = BertModel.from_pretrained(biobert_pmc_path)
#bert_model_pubmed = BertModel.from_pretrained(biobert_pubmed_path)
#bert_model_pubmed_pmc = BertModel.from_pretrained(biobert_pubmed_pmc_path)

<a id="vocab"></a>
### Buildling vocabularies
Normalizing case, lemmatization, stemming, removing stopwords, removing punctuation, handling numerics, creating parse trees, part-of-speech tagging, and anything else necessary for a particular dataset of descripitons.

In [69]:
# Constructing a vocabulary by looking at what words are overrepresented in domain specific text.
background_corpus = open(background_corpus_filename,"r").read()
phenotypes_corpus = open(phenotypes_corpus_filename,"r").read()
tokens = get_overrepresented_tokens(phenotypes_corpus, background_corpus, max_features=5000)
vocabulary_from_text = build_vocabulary_from_tokens(tokens)

# Constructing a vocabulary by assuming all words present in a given ontology are important.
ontology = Ontology(ontology_filename)
vocabulary_from_ontology = build_vocabulary_from_tokens(ontology.get_tokens())

<a id="preprocessing"></a>
### Preprocessing text descriptions
The preprocessing methods applied to the phenotype descriptions are a choice which impacts the subsequent vectorization and similarity methods which construct the pairwise distance matrix from each of these descriptions. The preprocessing methods that make sense are also highly dependent on the vectorization method or embedding method that is to be applied. For example, stemming (which is part of the full proprocessing done below using `gensim.preprocess_string` is useful for the n-grams and bag-of-words methods but not for the document embeddings methods which need each token to be in the vocabulary that was constructed and used when the model was trained. For this reason, embedding methods with pretrained models where the vocabulary is fixed should have a lighter degree of preprocessing not involving stemming or lemmatization but should involve things like removal of non-alphanumerics and normalizing case. 

In [70]:
# Obtain a mapping between IDs and the raw text descriptions associated with that ID from the dataset.
descriptions = dataset.get_description_dictionary()

# Preprocessing of the text descriptions. Different methods are necessary for different approaches.
descriptions_full_preprocessing = {i:" ".join(preprocess_string(d)) for i,d in descriptions.items()}
descriptions_simple_preprocessing = {i:" ".join(simple_preprocess(d)) for i,d in descriptions.items()}
descriptions_no_stopwords = {i:remove_stopwords(d) for i,d in descriptions.items()}

# Generating random descriptions that are drawn from same token population and retain original lengths.
#tokens = [w for w in itertools.chain.from_iterable(word_tokenize(desc) for desc in descriptions.values())]
#descriptions_scrambled = {k:" ".join(np.random.choice(tokens,len(word_tokenize(v)))) for k,v in descriptions.items()}

<a id="pos_tagging"></a>
### POS tagging the phenotype descriptions for nouns and adjectives
Note that preprocessing of the descriptions should be done after part-of-speech tagging, because tokens that are removed during preprocessing before n-gram analysis contain information that the parser needs to accurately call parts-of-speech. This step should be done on the raw descriptions and then the resulting bags of words can be subset using additional preprocesssing steps before input in one of the vectorization methods.

In [71]:
get_pos_tokens = lambda text,pos: " ".join([t[0] for t in nltk.pos_tag(word_tokenize(text)) if t[1].lower()==pos.lower()])
descriptions_noun_only =  {i:get_pos_tokens(d,"NN") for i,d in descriptions.items()}
descriptions_noun_only_full_preprocessing = {i:" ".join(preprocess_string(d)) for i,d in descriptions_noun_only.items()}
descriptions_noun_only_simple_preprocessing = {i:" ".join(simple_preprocess(d)) for i,d in descriptions_noun_only.items()}
descriptions_adj_only =  {i:get_pos_tokens(d,"JJ") for i,d in descriptions.items()}
descriptions_adj_only_full_preprocessing = {i:" ".join(preprocess_string(d)) for i,d in descriptions_adj_only.items()}
descriptions_adj_only_simple_preprocessing = {i:" ".join(simple_preprocess(d)) for i,d in descriptions_adj_only.items()}

<a id="annotation"></a>
### Annotating descriptions with ontology terms
This section generates dictionaries that map gene IDs from the dataset to lists of strings, where those strings are ontology term IDs. How the term IDs are found for each gene entry with its corresponding phenotype description depends on the cell below. Firstly, the terms are found by using the NOBLE Coder annotation tool through these wrapper functions to identify the terms by looking for instances of the term's label or synonyms in the actual text of the phenotype descriptions. Secondly, the next cell just draws the terms directly from the dataset itself. In this case, these are high-confidence annotations done by curators for a comparison against what can be accomplished through computational analysis of the text.

In [ ]:
# Run the ontology term annotators over either raw or preprocessed text descriptions.
annotations_noblecoder_precise = annotate_using_noble_coder(descriptions, noblecoder_jarfile_path, "mo", precise=1)
annotations_noblecoder_partial = annotate_using_noble_coder(descriptions, noblecoder_jarfile_path, "mo", precise=0)

In [ ]:
# Get the ID to term list annotation dictionaries for each ontology in the dataset.
annotations = dataset.get_annotations_dictionary()
go_annotations = {k:[term for term in v if term[0:2]=="GO"] for k,v in annotations.items()}
po_annotations = {k:[term for term in v if term[0:2]=="PO"] for k,v in annotations.items()}

<a id="matrix"></a>
<a id="methods"></a>
<a id="run"></a>
<a id="merging"></a>
### Generating vector representations and pairwise distances matrices
This section uses the text descriptions, preprocessed text descriptions, or ontology term annotations created or read in the previous sections to generate a vector representation for each gene and build a pairwise distance matrix for the whole dataset. Each method specified is a unique combination of a method of vectorization (bag-of-words, n-grams, document embedding model, etc) and distance metric (Euclidean, Jaccard, cosine, etc) applied to those vectors in constructing the pairwise matrix. The method of vectorization here is equivalent to feature selection, so the task is to figure out which type of vectors will encode features that are useful (n-grams, full words, only words from a certain vocabulary, etc).

In [72]:
# Define a list of different methods for calculating distance between text descriptions using the Methods object 
# defined in the utilities for this notebook. The constructor takes a string for the method name, a string defining
# the hyperparameter choices for that method, a function to be called to run this method, a dictionary of arguments
# by keyword that should be passed to that function, and a distance metric from scipy.spatial.distance to associate
# with this method.

methods = [

    
    # Methods that use neural networks to generate embeddings.
    #Method("Doc2Vec Wikipedia", "Size=300", pw.pairwise_square_doc2vec, {"model":doc2vec_wiki_model, "ids_to_texts":descriptions, "metric":"cosine"}, spatial.distance.cosine),
    #Method("Doc2Vec PubMed", "Size=100", pw.pairwise_square_doc2vec, {"model":doc2vec_pubmed_model, "ids_to_texts":descriptions, "metric":"cosine"}, spatial.distance.cosine),
    Method("Word2Vec Wikipedia", "Size=300,Mean", pw.pairwise_square_word2vec, {"model":word2vec_model, "ids_to_texts":descriptions, "metric":"cosine", "method":"mean"}, spatial.distance.cosine),
    Method("Word2Vec Wikipedia", "Size=300,Max", pw.pairwise_square_word2vec, {"model":word2vec_model, "ids_to_texts":descriptions, "metric":"cosine", "method":"max"}, spatial.distance.cosine),
    #Method("BERT", "Base:Layers=2,Concatenated", pw.pairwise_square_bert, {"model":bert_model_base, "tokenizer":bert_tokenizer_base, "ids_to_texts":descriptions, "metric":"cosine", "method":"concat", "layers":2}, spatial.distance.cosine),
    #Method("BERT", " Base:Layers=3,Concatenated", pw.pairwise_square_bert, {"model":bert_model_base, "tokenizer":bert_tokenizer_base, "ids_to_texts":descriptions, "metric":"cosine", "method":"concat", "layers":3}, spatial.distance.cosine),
    #Method("BERT", " Base:Layers=4,Concatenated", pw.pairwise_square_bert, {"model":bert_model_base, "tokenizer":bert_tokenizer_base, "ids_to_texts":descriptions, "metric":"cosine", "method":"concat", "layers":4}, spatial.distance.cosine),
    #Method("BERT", " Base:Layers=2,Summed", pw.pairwise_square_bert, {"model":bert_model_base, "tokenizer":bert_tokenizer_base, "ids_to_texts":descriptions, "metric":"cosine", "method":"sum", "layers":2}, spatial.distance.cosine),
    #Method("BERT", " Base:Layers=3,Summed", pw.pairwise_square_bert, {"model":bert_model_base, "tokenizer":bert_tokenizer_base, "ids_to_texts":descriptions, "metric":"cosine", "method":"sum", "layers":3}, spatial.distance.cosine),
    #Method("BERT", " Base:Layers=4,Summed", pw.pairwise_square_bert, {"model":bert_model_base, "tokenizer":bert_tokenizer_base, "ids_to_texts":descriptions, "metric":"cosine", "method":"sum", "layers":4}, spatial.distance.cosine),
    #Method("BioBERT", "PMC,Layers=2,Concatenated", pw.pairwise_square_bert, {"model":bert_model_pmc, "tokenizer":bert_tokenizer_pmc, "ids_to_texts":descriptions, "metric":"cosine", "method":"concat", "layers":2}, spatial.distance.cosine),
    #Method("BioBERT", "PMC,Layers=3,Concatenated", pw.pairwise_square_bert, {"model":bert_model_pmc, "tokenizer":bert_tokenizer_pmc, "ids_to_texts":descriptions, "metric":"cosine", "method":"concat", "layers":3}, spatial.distance.cosine),
    #Method("BioBERT", "PMC,Layers=4,Concatenated", pw.pairwise_square_bert, {"model":bert_model_pmc, "tokenizer":bert_tokenizer_pmc, "ids_to_texts":descriptions, "metric":"cosine", "method":"concat", "layers":4}, spatial.distance.cosine),
    #Method("BioBERT", "PubMed,Layers=4,Concatenated", pw.pairwise_square_bert, {"model":bert_model_pubmed, "tokenizer":bert_tokenizer_pubmed, "ids_to_texts":descriptions, "metric":"cosine", "method":"concat", "layers":4}, spatial.distance.cosine),
    #Method("BioBERT", "PubMed,PMC,Layers=4,Concatenated", pw.pairwise_square_bert, {"model":bert_model_pubmed_pmc, "tokenizer":bert_tokenizer_pubmed_pmc, "ids_to_texts":descriptions, "metric":"cosine", "method":"concat", "layers":4}, spatial.distance.cosine),
        
    # Methods that use variations on the n-grams approach with full preprocessing (includes stemming).
    Method("N-Grams", "Full,Words,1-grams,2-grams", pw.pairwise_square_ngrams, {"ids_to_texts":descriptions_full_preprocessing, "metric":"cosine", "binary":False, "analyzer":"word", "ngram_range":(1,2),"max_features":10000, "tfidf":False}, spatial.distance.cosine),
    Method("N-Grams", "Full,Words,1-grams,2-grams,Binary", pw.pairwise_square_ngrams, {"ids_to_texts":descriptions_full_preprocessing, "metric":"jaccard", "binary":True, "analyzer":"word", "ngram_range":(1,2), "max_features":10000, "tfidf":False}, spatial.distance.jaccard),
    Method("N-Grams", "Full,Words,1-grams", pw.pairwise_square_ngrams, {"ids_to_texts":descriptions_full_preprocessing, "metric":"cosine", "binary":False, "analyzer":"word", "ngram_range":(1,1), "max_features":10000, "tfidf":False}, spatial.distance.cosine),
    Method("N-Grams", "Full,Words,1-grams,Binary", pw.pairwise_square_ngrams, {"ids_to_texts":descriptions_full_preprocessing, "metric":"jaccard", "binary":True, "analyzer":"word", "ngram_range":(1,1), "max_features":10000, "tfidf":False}, spatial.distance.jaccard),
    Method("N-Grams", "Full,Words,1-grams,2-grams,TFIDF", pw.pairwise_square_ngrams, {"ids_to_texts":descriptions_full_preprocessing, "metric":"cosine", "binary":False, "analyzer":"word", "ngram_range":(1,2),"max_features":10000, "tfidf":True}, spatial.distance.cosine),
    Method("N-Grams", "Full,Words,1-grams,2-grams,Binary,TFIDF", pw.pairwise_square_ngrams, {"ids_to_texts":descriptions_full_preprocessing, "metric":"cosine", "binary":True, "analyzer":"word", "ngram_range":(1,2), "max_features":10000, "tfidf":True}, spatial.distance.cosine),
    Method("N-Grams", "Full,Words,1-grams,TFIDF", pw.pairwise_square_ngrams, {"ids_to_texts":descriptions_full_preprocessing, "metric":"cosine", "binary":False, "analyzer":"word", "ngram_range":(1,1), "max_features":10000, "tfidf":True}, spatial.distance.cosine),
    Method("N-Grams", "Full,Words,1-grams,Binary,TFIDF", pw.pairwise_square_ngrams, {"ids_to_texts":descriptions_full_preprocessing, "metric":"cosine", "binary":True, "analyzer":"word", "ngram_range":(1,1), "max_features":10000, "tfidf":True}, spatial.distance.cosine),
    
    # Methods that use variations on the n-grams approach with simple preprocessing (no stemming).
    Method("N-Grams", "Simple,Words,1-grams,2-grams", pw.pairwise_square_ngrams, {"ids_to_texts":descriptions_simple_preprocessing, "metric":"cosine", "binary":False, "analyzer":"word", "ngram_range":(1,2),"max_features":10000, "tfidf":False}, spatial.distance.cosine),
    Method("N-Grams", "Simple,Words,1-grams,2-grams,Binary", pw.pairwise_square_ngrams, {"ids_to_texts":descriptions_simple_preprocessing, "metric":"jaccard", "binary":True, "analyzer":"word", "ngram_range":(1,2), "max_features":10000, "tfidf":False}, spatial.distance.jaccard),
    Method("N-Grams", "Simple,Words,1-grams", pw.pairwise_square_ngrams, {"ids_to_texts":descriptions_simple_preprocessing, "metric":"cosine", "binary":False, "analyzer":"word", "ngram_range":(1,1), "max_features":10000, "tfidf":False}, spatial.distance.cosine),
    Method("N-Grams", "Simple,Words,1-grams,Binary", pw.pairwise_square_ngrams, {"ids_to_texts":descriptions_simple_preprocessing, "metric":"jaccard", "binary":True, "analyzer":"word", "ngram_range":(1,1), "max_features":10000, "tfidf":False}, spatial.distance.jaccard),
    Method("N-Grams", "Simple,Words,1-grams,2-grams,TFIDF", pw.pairwise_square_ngrams, {"ids_to_texts":descriptions_simple_preprocessing, "metric":"cosine", "binary":False, "analyzer":"word", "ngram_range":(1,2),"max_features":10000, "tfidf":True}, spatial.distance.cosine),
    Method("N-Grams", "Simple,Words,1-grams,2-grams,Binary,TFIDF", pw.pairwise_square_ngrams, {"ids_to_texts":descriptions_simple_preprocessing, "metric":"cosine", "binary":True, "analyzer":"word", "ngram_range":(1,2), "max_features":10000, "tfidf":True}, spatial.distance.cosine),
    Method("N-Grams", "Simple,Words,1-grams,TFIDF", pw.pairwise_square_ngrams, {"ids_to_texts":descriptions_simple_preprocessing, "metric":"cosine", "binary":False, "analyzer":"word", "ngram_range":(1,1), "max_features":10000, "tfidf":True}, spatial.distance.cosine),
    Method("N-Grams", "Simple,Words,1-grams,Binary,TFIDF", pw.pairwise_square_ngrams, {"ids_to_texts":descriptions_simple_preprocessing, "metric":"cosine", "binary":True, "analyzer":"word", "ngram_range":(1,1), "max_features":10000, "tfidf":True}, spatial.distance.cosine),
    
    # Methods that use variations on the n-grams approach selecting for specific parts-of-speech.
    Method("N-Grams", "Full,Nouns,1-grams", pw.pairwise_square_ngrams, {"ids_to_texts":descriptions_noun_only_full_preprocessing, "metric":"cosine", "binary":False, "analyzer":"word", "ngram_range":(1,1), "max_features":10000, "tfidf":False}, spatial.distance.cosine),
    Method("N-Grams", "Full,Nouns,1-grams,Binary", pw.pairwise_square_ngrams, {"ids_to_texts":descriptions_noun_only_full_preprocessing, "metric":"jaccard", "binary":True, "analyzer":"word", "ngram_range":(1,1), "max_features":10000, "tfidf":False}, spatial.distance.jaccard),
    Method("N-Grams", "Full,Nouns,1-grams,TFIDF", pw.pairwise_square_ngrams, {"ids_to_texts":descriptions_noun_only_full_preprocessing, "metric":"cosine", "binary":False, "analyzer":"word", "ngram_range":(1,1), "max_features":10000, "tfidf":True}, spatial.distance.cosine),
    Method("N-Grams", "Full,Nouns,1-grams,Binary,TFIDF", pw.pairwise_square_ngrams, {"ids_to_texts":descriptions_noun_only_full_preprocessing, "metric":"cosine", "binary":True, "analyzer":"word", "ngram_range":(1,1), "max_features":10000, "tfidf":True}, spatial.distance.cosine),
    Method("N-Grams", "Full,Adjectives,1-grams", pw.pairwise_square_ngrams, {"ids_to_texts":descriptions_adj_only_full_preprocessing, "metric":"cosine", "binary":False, "analyzer":"word", "ngram_range":(1,1), "max_features":10000, "tfidf":False}, spatial.distance.cosine),
    Method("N-Grams", "Full,Adjectives,1-grams,Binary", pw.pairwise_square_ngrams, {"ids_to_texts":descriptions_adj_only_full_preprocessing, "metric":"jaccard", "binary":True, "analyzer":"word", "ngram_range":(1,1), "max_features":10000, "tfidf":False}, spatial.distance.jaccard),
    Method("N-Grams", "Full,Adjectives,1-grams,TFIDF", pw.pairwise_square_ngrams, {"ids_to_texts":descriptions_adj_only_full_preprocessing, "metric":"cosine", "binary":False, "analyzer":"word", "ngram_range":(1,1), "max_features":10000, "tfidf":True}, spatial.distance.cosine),
    Method("N-Grams", "Full,Adjectives,1-grams,Binary,TFIDF", pw.pairwise_square_ngrams, {"ids_to_texts":descriptions_adj_only_full_preprocessing, "metric":"cosine", "binary":True, "analyzer":"word", "ngram_range":(1,1), "max_features":10000, "tfidf":True}, spatial.distance.cosine),
    
    
    # Methods that use terms inferred from automated annotation of the text.
    #Method("NOBLE Coder", "Precise", pw.pairwise_square_annotations, {"ids_to_annotations":annotations_noblecoder_precise, "ontology":ontology, "binary":True, "metric":"jaccard", "tfidf":False}, spatial.distance.jaccard),
    #Method("NOBLE Coder", "Partial", pw.pairwise_square_annotations, {"ids_to_annotations":annotations_noblecoder_partial, "ontology":ontology, "binary":True, "metric":"jaccard", "tfidf":False}, spatial.distance.jaccard),
    #Method("NOBLE Coder", "Precise,TFIDF", pw.pairwise_square_annotations, {"ids_to_annotations":annotations_noblecoder_precise, "ontology":ontology, "binary":True, "metric":"cosine", "tfidf":True}, spatial.distance.cosine),
    #Method("NOBLE Coder", "Partial,TFIDF", pw.pairwise_square_annotations, {"ids_to_annotations":annotations_noblecoder_partial, "ontology":ontology, "binary":True, "metric":"cosine", "tfidf":True}, spatial.distance.cosine),
    
    # Methods that use terms assigned by humans that are present in the dataset.
    #Method("GO", "Default", pw.pairwise_square_annotations, {"ids_to_annotations":go_annotations, "ontology":ontology, "metric":"jaccard", "binary":True, "analyzer":"word", "ngram_range":(1,1), "tfidf":False}, spatial.distance.jaccard),
    #Method("PO", "Default", pw.pairwise_square_annotations, {"ids_to_annotations":po_annotations, "ontology":ontology, "metric":"jaccard", "binary":True, "analyzer":"word", "ngram_range":(1,1), "tfidf":False}, spatial.distance.jaccard),


]

# Generate all of the pairwise distance matrices in parallel.
start_time_mp = time.perf_counter()
pool = mp.Pool(mp.cpu_count())
results = [pool.apply_async(function_wrapper_with_duration, args=(method.function, method.kwargs)) for method in methods]
results = [result.get() for result in results]
graphs = {method.name_with_hyperparameters:result[0] for method,result in zip(methods,results)}
metric_dict = {method.name_with_hyperparameters:method.metric for tup in methods}
durations = {method.name_with_hyperparameters:result[1] for method,result in zip(methods,results)}
pool.close()
pool.join()    
total_time_mp = time.perf_counter()-start_time_mp

# Reporting how long each matrix took to build and how much time parallel processing saved.
print("Durations of generating each pairwise similarity matrix (hh:mm:ss)")
print("-----------------------------------------------------------------")
savings = total_time_mp/sum(durations.values())
for (name,duration) in durations.items():
    print("{:50} {}".format(name, to_hms(duration)))
print("-----------------------------------------------------------------")
print("{:15} {}".format("total", to_hms(sum(durations.values()))))
print("{:15} {} ({:.2%} of single thread time)".format("multiprocess", to_hms(total_time_mp), savings))

In [73]:
# Generate all the pairwise distance matrices (not in parallel).
graphs = {}
names = []
durations = []
for method in methods:
    graph,duration = function_wrapper_with_duration(function=method.function, args=method.kwargs)
    graphs[method.name_with_hyperparameters] = graph
    names.append(method.name_with_hyperparameters)
    durations.append(to_hms(duration))
    print("{:50} {}".format(method.name_with_hyperparameters,to_hms(duration)))
durations_df = pd.DataFrame({"method":names,"duration":durations})
durations_df.to_csv(os.path.join(OUTPUT_DIR,"durations.csv"), index=False)

Word2Vec Wikipedia:Size=300,Mean                   00:00:02
Word2Vec Wikipedia:Size=300,Max                    00:00:02
N-Grams:Full,Words,1-grams,2-grams                 00:00:41


KeyboardInterrupt: 

In [74]:
# Merging all of the edgelist dataframes together.
metric_dict = {method.name_with_hyperparameters:method.metric for method in methods}
methods = list(graphs.keys())
edgelists = {k:v.edgelist for k,v in graphs.items()}
df = pw.merge_edgelists(edgelists, default_value=0.000)
df = pw.remove_self_loops(df)
df.tail(10)

,from,to,"Word2Vec Wikipedia:Size=300,Mean","Word2Vec Wikipedia:Size=300,Max","N-Grams:Full,Words,1-grams,2-grams"
3270389,6024,6025,0.087342,0.053785,0.855134
3270390,6024,6026,0.150344,0.189917,0.632311
3270391,6024,6027,0.111915,0.059905,0.790042
3270392,6024,6028,0.113737,0.059905,0.815636
3270394,6025,6026,0.291678,0.229865,0.735151
3270395,6025,6027,0.082383,0.057436,0.765190
3270396,6025,6028,0.086275,0.057436,0.785179
3270398,6026,6027,0.273921,0.187227,0.702469
3270399,6026,6028,0.281280,0.187227,0.772890
3270401,6027,6028,0.001311,0.000000,0.018594


### Finding k nearest neighbors through each method over this dataset.

In [75]:
some_text = "plants are reduced in height"
for method in methods:
    g = graphs[method]
    nn = g.get_nearest_neighbor_ids(some_text, k=10)
    print(nn)

[3043, 4350, 5969, 3922, 3039, 5839, 4349, 3419, 3500, 3460]
[3042, 5951, 5558, 638, 5275, 3882, 3460, 5487, 3546, 3141]
[3145, 3729, 1393, 930, 3144, 4305, 5512, 3175, 4761, 3604]


### Collapsing vocabulary with distance matrix between tokens and doing topic modeling.

In [ ]:
tokens = list(set([w for w in flatten(word_tokenize(d) for d in descriptions_simple_preprocessing.values())]))
tokens_dict = {i:w for i,w in enumerate(tokens)}
graph = pw.pairwise_square_word2vec(word2vec_model, tokens_dict, "cosine")

# Get the objects needed to collapse the vocabulary and make a new vocabulary.
# Make sure that the tokens list is in the same order as the indices representing each word in the distance matrix.
# This is only trivial here because the IDs used are ordered integers 0 to n, but this might not always be the case.
distance_matrix = graph.array
tokens = [tokens_dict[graph.row_index_to_id[index]] for index in np.arange(distance_matrix.shape[0])]
threshold = 0.3
vocab, transform, untransform =  collapse_vocabulary_by_distance(tokens, distance_matrix, threshold)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF

# Some notes about this, making sure the vectorizer has the lowercase=False option, to retain groups at TOKEN_.
# LDA should be run on the independent phenes, not the phenotypes?
# Then the phenotypes themselves can be categorized as coming from 



# The list of texts has to be created from the same descriptions used to create the reduced vocab (simple preprocess).
to_reduced_vocab = lambda t: " ".join([transform.get(w,"") for w in t.split(" ")])
texts = [to_reduced_vocab(d) for d in descriptions_simple_preprocessing.values()]


vectorizer = TfidfVectorizer(max_features=10000, stop_words="english", max_df=0.95, min_df=2, lowercase=False)
features = vectorizer.fit_transform(texts)


cls = NMF(n_components=10, random_state=0)
cls.fit(features)


n_top_words=5
feature_names = vectorizer.get_feature_names()
for i,topic_vec in enumerate(cls.components_):
    print(i,end=" ")
    for fid in topic_vec.argsort()[-1:-n_top_words-1:-1]:
        
        word = feature_names[fid]
        #word = untransform[word]
        word = " ".join(untransform[word])
        
        
        print(word, end=' ')
    print()


    
# What's the fraction of each topic for a new phrase? (dimensionality reduction)


something = texts[0]
v = vectorizer.transform([something]).toarray()[0]
topic_vector = cls.transform([v]) # one value per topic
print(topic_vector)
print(topic_vector.argsort(axis=1)[0][-5:]) # What are the top five topics for this new sentence?







    







In [ ]:




def _infer_document_vector_from_lda(text, model, vectorizer):
    """ 
    Text has to be preprocessed in the same way as the text used to fit the model.
    So this includes things like replacing words with custom tokens to reduce the vocabulary size.
    """
    
    
    
    
    
    
    

In [ ]:
g = graphs["N-Grams:Full,Nouns,1-grams,Binary"]
identifier_list = list(g.vector_dictionary.keys())
m = np.array([g.vector_dictionary[i] for i in identifier_list])


cls = NMF(n_components=30, random_state=0)
cls.fit(m)
n_top_words=5
feature_names = g.vectorizer_object.get_feature_names()
for i,topic_vec in enumerate(cls.components_):
    print(i,end=" ")
    for fid in topic_vec.argsort()[-1:-n_top_words-1:-1]:
        print(feature_names[fid], end=' ')
    print()

    
    
# What's the fraction of each topic for a new phrase? (dimensionality reduction)
v = g.get_vector("stress freez acclim salt germin embryo arrest stage growth lethal ")
topic_vector = cls.transform([v]) # one value per topic

print(topic_vector)
print(topic_vector.argsort(axis=1)[0][-5:]) # What are the top five topics for this new sentence?

In [ ]:
v,td,utd = collapse_vocabulary_by_distance(tokens, distance_matrix, 0.5124)
print(len(v))

In [ ]:
from sklearn.cluster import AffinityPropagation

# Create the n by n distance matrix. 
m = graphs["N-Grams:Full,Words,1-grams,2-grams"].array
to_id = graphs["N-Grams:Full,Words,1-grams,2-grams"].row_index_to_id
clustering = AffinityPropagation().fit(m)
for c in pd.unique(clustering.labels_):
    print("CLUSTER",c)
    print()
    for i,label in enumerate(clustering.labels_):
        if (label == c):
            print(descriptions[to_id[i]])
            print()


In [ ]:
from sklearn.cluster import AgglomerativeClustering

# Create the n by n distance matrix. 
m = graphs["N-Grams:Full,Words,1-grams,2-grams"].array
to_id = graphs["N-Grams:Full,Words,1-grams,2-grams"].row_index_to_id
clustering = AgglomerativeClustering(n_clusters=10, linkage="complete", affinity="precomputed").fit(m)
for c in pd.unique(clustering.labels_):
    print("CLUSTER",c)
    print()
    for i,label in enumerate(clustering.labels_):
        if (label == c):
            print(descriptions[to_id[i]])
            print()








In [ ]:
# temporary, the clustering stuff.
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=10, random_state=0, )



from sklearn.neighbors import NearestNeighbors
knn = NearestNeighbors(n_neighbors=2, metric="cosine")

g = graphs["N-Grams:Full,Words,1-grams,2-grams,Binary"]

ids = list(g.vector_dictionary.keys())
m = np.array([g.vector_dictionary[i] for i in ids])


knn.fit(m)



query_text = "one leaf was green"
query_features = g.vectorizer_object.transform([query_text]).toarray()
print(m.shape)
print(query_features.shape)


In [ ]:
neighbor_indices = knn.kneighbors(query_features, return_distance=False)[0]
print(neighbor_indices)
for i in neighbor_indices:
    actual_id = ids[i]
    print(descriptions[actual_id])

In [ ]:
# topic modelling?
from sklearn.decomposition import NMF

g = graphs["N-Grams:Full,Nouns,1-grams,Binary"]
identifier_list = list(g.vector_dictionary.keys())
m = np.array([g.vector_dictionary[i] for i in identifier_list])


cls = NMF(n_components=30, random_state=0)
cls.fit(m)
n_top_words=5
feature_names = g.vectorizer_object.get_feature_names()
for i,topic_vec in enumerate(cls.components_):
    print(i,end=" ")
    for fid in topic_vec.argsort()[-1:-n_top_words-1:-1]:
        print(feature_names[fid], end=' ')
    print()

    
    
# What's the fraction of each topic for a new phrase? (dimensionality reduction)
v = g.get_vector("stress freez acclim salt germin embryo arrest stage growth lethal ")
topic_vector = cls.transform([v]) # one value per topic

print(topic_vector)
print(topic_vector.argsort(axis=1)[0][-5:]) # What are the top five topics for this new sentence?

### Option 1: Merging in the previously curated similarity values from the Oellrich, Walls et al. (2015) dataset
This section reads in a file that contains the previously calculated distance values from the Oellrich, Walls et al. (2015) dataset, and merges it with the values which are obtained here for all of the applicable natural language processing or machine learning methods used, so that the graphs which are specified by these sets of distances values can be evaluated side by side in the subsequent sections.

In [ ]:
# Add a column that indicates the distance estimated using curated EQ statements.
df = df.merge(right=pppn_edgelist.df, how="left", on=["from","to"])
df.fillna(value=0.000,inplace=True)
df.rename(columns={"value":"EQs"}, inplace=True)
df["EQs"] = 1-df["EQs"]
methods.append("EQs")
df.tail(10)

### Option 2: Merging with information about shared pathways or groupings.
The relevant information for each edge includes questions like whether or not the two genes that edge connects share a group or biochemical pathway in common, or if those genes are from the same species. This information can then later be used as the target values for predictive models, or for filtering the graphs represented by these edge lists. Either the grouping information or the protein-protein interaction information should be used.

In [76]:
# Generate a column indicating whether or not the two genes share this features (e.g., pathway in common, same group).
df["shared"] = df[["from","to"]].apply(lambda x: len(set(id_to_group_ids[x["from"]]).intersection(set(id_to_group_ids[x["to"]])))>0, axis=1)*1
counts = Counter(df["shared"].values)
print("{} edges between genes that share this feature, {} edges between genes that do not.".format(counts.get(1,0),counts.get(0,0)))

# Generate a column indicating whether or not the two genes are from the same species.
species_dict = dataset.get_species_dictionary()
df["same"] = df[["from","to"]].apply(lambda x: species_dict[x["from"]]==species_dict[x["to"]],axis=1)*1
counts = Counter(df["same"].values)
print("{} edges between genes from the same species, {} edges between genes that are not.".format(counts.get(1,0),counts.get(0,0)))

367416 edges between genes that share this feature, 2900430 edges between genes that do not.
3267846 edges between genes from the same species, 0 edges between genes that are not.


### Option 3: Merging with information about protein-protein interactions.

In [ ]:
# Merging information from the protein-protein interaction database with this dataset.
df = df.merge(right=string_data.df, how="left", on=["from","to"])
df.fillna(value=0,inplace=True)
df["shared"] = (df["combined_score"] != 0.00)*1
df.tail(12)

<a id="ensemble"></a>
### Combining multiple distance methods with machine learning models
The purpose of this section is to iteratively train models on subsections of the dataset using simple regression or machine learning approaches to predict a value from zero to one indicating indicating how likely is it that two genes share atleast one of the specified groups in common. The information input to these models is the distance scores provided by each method in some set of all the methods used in this notebook. The purpose is to see whether or not a function of these similarity scores specifically trained to the task of predicting common groupings is better able to used the distance metric information to report a score for this task.

In [ ]:
# Iteratively create models for combining output values from multiple semantic similarity methods.
method = "Logistic Regression"
splits = 12
kf = KFold(n_splits=splits, random_state=14271, shuffle=True)
df[method] = pd.Series()
for train,test in kf.split(df):
    lr_model = train_logistic_regression_model(df=df.iloc[train], predictor_columns=methods, target_column="shared")
    df[method].iloc[test] = apply_logistic_regression_model(df=df.iloc[test], predictor_columns=methods, model=lr_model)
df[method] = 1-df[method]
methods.append(method)

In [ ]:
# Iteratively create models for combining output values from multiple semantic similarity methods.
method = "Random Forest"
splits = 2
kf = KFold(n_splits=splits, random_state=14271, shuffle=True)
df[method] = pd.Series()
for train,test in kf.split(df):
    rf_model = train_random_forest_model(df=df.iloc[train], predictor_columns=methods, target_column="shared")
    df[method].iloc[test] = apply_random_forest_model(df=df.iloc[test],predictor_columns=methods, model=rf_model)
df[method] = 1-df[method]
methods.append(method)

In [ ]:
"""
# Don't do it iteratively, just treat this entire dataset as training data and save the models.
# Logistic Regression
lr_name = "Logistic Regression"
model = train_logistic_regression_model(df=df, predictor_columns=methods, target_column="shared")
df[lr_name] = apply_logistic_regression_model(df=df, predictor_columns=methods, model=model)
df[lr_name] = 1-df[lr_name]
save_to_pickle(obj=model,path="../data/pickles/lr.model")
# Random Forest
rf_name = "Random Forest"
model = train_random_forest_model(df=df, predictor_columns=methods, target_column="shared")
df[rf_name] = apply_random_forest_model(df=df, predictor_columns=methods, model=model)
df[rf_name] = 1-df[rf_name]
save_to_pickle(obj=model,path="../data/pickles/rf.model")
# Add these methods to the method names to use for the below analysis.
methods.append(lr_name)
methods.append(rf_name)
"""

<a id="ks"></a>
### Do the edges joining genes that share atleast one pathway come from a different distribution?
The purpose of this section is to visualize kernel estimates for the distributions of distance or similarity scores generated by each of the methods tested for measuring semantic similarity or generating vector representations of the phenotype descriptions. Ideally, better methods should show better separation betwene the distributions for distance values between two genes involved in a common specified group or two genes that are not. Additionally, a statistical test is used to check whether these two distributions are significantly different from each other or not, although this is a less informative measure than the other tests used in subsequent sections, because it does not address how useful these differences in the distributions actually are for making predictions about group membership.

In [ ]:
# Use Kolmogorov-Smirnov test to see if edges between genes that share a group come from a distinct distribution.
ppi_pos_dict = {name:(df[df["shared"] > 0.00][name].values) for name in methods}
ppi_neg_dict = {name:(df[df["shared"] == 0.00][name].values) for name in methods}
for name in methods:
    stat,p = ks_2samp(ppi_pos_dict[name],ppi_neg_dict[name])
    pos_mean = np.average(ppi_pos_dict[name])
    neg_mean = np.average(ppi_neg_dict[name])
    pos_n = len(ppi_pos_dict[name])
    neg_n = len(ppi_neg_dict[name])
    TABLE[name].update({"mean_1":pos_mean, "mean_0":neg_mean, "n_1":pos_n, "n_0":neg_n})
    TABLE[name].update({"ks":stat, "ks_pval":p})
    
    
# Show the kernel estimates for each distribution of weights for each method.
num_plots, plots_per_row, row_width, row_height = (len(methods), 4, 14, 3)
fig,axs = plt.subplots(math.ceil(num_plots/plots_per_row), plots_per_row, squeeze=False)
for name,ax in zip(methods,axs.flatten()):
    ax.set_title(name)
    ax.set_xlabel("value")
    ax.set_ylabel("density")
    sns.kdeplot(ppi_pos_dict[name], color="black", shade=False, alpha=1.0, ax=ax)
    sns.kdeplot(ppi_neg_dict[name], color="black", shade=True, alpha=0.1, ax=ax) 
fig.set_size_inches(row_width, row_height*math.ceil(num_plots/plots_per_row))
fig.tight_layout()
fig.savefig(os.path.join(OUTPUT_DIR,"kernel_density.png"),dpi=400)
plt.close()

<a id="within"></a>
### Looking at within-group or within-pathway distances in each graph
The purpose of this section is to determine which methods generated graphs which tightly group genes which share common pathways or group membership with one another. In order to compare across different methods where the distance value distributions are different, the mean distance values for each group for each method are convereted to percentile scores. Lower percentile scores indicate that the average distance value between any two genes that belong to that group is lower than most of the distance values in the entire distribution for that method.

In [77]:
# Get all the average within-pathway phenotype distance values for each method for each particular pathway.
group_id_to_ids = groups.get_group_id_to_ids_dict(dataset.get_gene_dictionary())
group_ids = list(group_id_to_ids.keys())
graph = IndexedGraph(df)
within_weights_dict = defaultdict(lambda: defaultdict(list))
within_percentiles_dict = defaultdict(lambda: defaultdict(list))
all_weights_dict = {}
for method in methods:
    all_weights_dict[method] = df[method].values
    for group in group_ids:
        within_ids = group_id_to_ids[group]
        within_pairs = [(i,j) for i,j in itertools.permutations(within_ids,2)]
        mean_weight = np.mean((graph.get_values(within_pairs, kind=method)))
        within_weights_dict[method][group] = mean_weight
        within_percentiles_dict[method][group] = stats.percentileofscore(df[method].values, mean_weight, kind="rank")

# Generating a dataframe of percentiles of the mean in-group distance scores.
heatmap_data = pd.DataFrame(within_percentiles_dict)
heatmap_data = heatmap_data.dropna(axis=0, inplace=False)
heatmap_data = heatmap_data.round(4)

# Adding relevant information to this dataframe and saving.
heatmap_data["mean_rank"] = heatmap_data.rank().mean(axis=1)
heatmap_data["mean_percentile"] = heatmap_data.mean(axis=1)
heatmap_data.sort_values(by="mean_percentile", inplace=True)
heatmap_data.reset_index(inplace=True)
heatmap_data["group_id"] = heatmap_data["index"]
heatmap_data["full_name"] = heatmap_data["group_id"].apply(lambda x: groups.get_long_name(x))
heatmap_data["n"] = heatmap_data["group_id"].apply(lambda x: len(group_id_to_ids[x]))
heatmap_data = heatmap_data[flatten(["group_id","full_name","n","mean_percentile","mean_rank",methods])]
heatmap_data.to_csv(os.path.join(OUTPUT_DIR,"within_distances.csv"), index=False)
heatmap_data.head(5)

,group_id,full_name,n,mean_percentile,mean_rank,"Word2Vec Wikipedia:Size=300,Mean","Word2Vec Wikipedia:Size=300,Max","N-Grams:Full,Words,1-grams,2-grams"
0,CDR,Circadian rhythms,13,6.207825,3.000000,9.6472,8.3347,3.8494
1,GEM,"Gametophyte, embryo defective (2-10% mutant se...",37,8.126050,2.000000,13.5858,15.7159,1.2025
2,NHM,No homozygous mutant plants; Cause not determined,18,8.523033,2.333333,14.0092,16.6486,1.1010
3,GAM,Gametophyte defective (<2% mutant seeds),82,10.537217,3.666667,17.0872,19.6647,1.7303
4,EMG,"Embryo, gametophyte defective (>10% mutant seeds)",60,14.571617,7.666667,18.6201,30.0952,1.9045


<a id="auc"></a>
### Predicting whether two genes belong to a common biochemical pathway
The purpose of this section is to see if whether or not two genes share atleast one common pathway can be predicted from the distance scores assigned using analysis of text similarity. The evaluation of predictability is done by reporting a precision and recall curve for each method, as well as remembering the area under the curve, and ratio between the area under the curve and the baseline (expected area when guessing randomly) for each method.

In [ ]:
y_true_dict = {name:df["shared"] for name in methods}
y_prob_dict = {name:(1 - df[name].values) for name in methods}
num_plots, plots_per_row, row_width, row_height = (len(methods), 4, 14, 3)
fig,axs = plt.subplots(math.ceil(num_plots/plots_per_row), plots_per_row, squeeze=False)
for method,ax in zip(methods, axs.flatten()):
    
    # Obtaining the values and metrics.
    y_true, y_prob = y_true_dict[method], y_prob_dict[method]
    n_pos, n_neg = Counter(y_true)[1], Counter(y_true)[0]
    precision, recall, thresholds = precision_recall_curve(y_true, y_prob)
    baseline = Counter(y_true)[1]/len(y_true) 
    area = auc(recall, precision)
    auc_to_baseline_auc_ratio = area/baseline
    TABLE[method].update({"auc":area, "baseline":baseline, "ratio":auc_to_baseline_auc_ratio})

    # Producing the precision recall curve.
    step_kwargs = ({'step': 'post'} if 'step' in signature(plt.fill_between).parameters else {})
    ax.step(recall, precision, color='black', alpha=0.2, where='post')
    ax.fill_between(recall, precision, alpha=0.7, color='black', **step_kwargs)
    ax.axhline(baseline, linestyle="--", color="lightgray")
    ax.set_xlabel('Recall')
    ax.set_ylabel('Precision')
    ax.set_ylim([0.0, 1.05])
    ax.set_xlim([0.0, 1.0])
    ax.set_title("PR {0} (Baseline={1:0.3f})".format(method, baseline))
    
fig.set_size_inches(row_width, row_height*math.ceil(num_plots/plots_per_row))
fig.tight_layout()
fig.savefig(os.path.join(OUTPUT_DIR,"prcurve_shared.png"),dpi=400)
plt.close()

<a id="y"></a>
### Are genes in the same biochemical pathway ranked higher with respect to individual nodes?
This is a way of statistically seeing if for some value k, the graph ranks more edges from some particular gene to any other gene that it has a true protein-protein interaction with higher or equal to rank k, than we would expect due to random chance. This way of looking at the problem helps to be less ambiguous than the previous methods, because it gets at the core of how this would actually be used. In other words, we don't really care how much true information we're missing as long as we're still able to pick up some new useful information by building these networks, so even though we could be missing a lot, what's going on at the very top of the results? These results should be comparable to very strictly thresholding the network and saying that the remaining edges are our guesses at interactions. This is comparable to just looking at the far left-hand side of the precision recall curves, but just quantifies it slightly differently.

In [ ]:
# When the edgelist is generated above, only the lower triangle of the pairwise matrix is retained for edges in the 
# graph. This means that in terms of the indices of each node, only the (i,j) node is listed in the edge list where
# i is less than j. This makes sense because the graph that's specified is assumed to already be undirected. However
# in order to be able to easily subset the edgelist by a single column to obtain rows that correspond to all edges
# connected to a particular node, this method will double the number of rows to include both (i,j) and (j,i) edges.
df = pw.make_undirected(df)

# What's the number of functional partners ranked k or higher in terms of phenotypic description similarity for 
# each gene? Also figure out the maximum possible number of functional partners that could be theoretically
# recovered in this dataset if recovered means being ranked as k or higher here.
k = 10      # The threshold of interest for gene ranks.
n = 100     # Number of Monte Carlo simulation iterations to complete.
df[list(methods)] = df.groupby("from")[list(methods)].rank()
ys = df[df["shared"]==1][list(methods)].apply(lambda s: len([x for x in s if x<=k]))
ymax = sum(df.groupby("from")["shared"].apply(lambda s: min(len([x for x in s if x==1]),k)))

# Monte Carlo simulation to see what the probability is of achieving each y-value by just randomly pulling k 
# edges for each gene rather than taking the top k ones that the similarity methods specifies when ranking.
ysims = [sum(df.groupby("from")["shared"].apply(lambda s: len([x for x in s.sample(k) if x>0.00]))) for i in range(n)]
for method in methods:
    pvalue = len([ysim for ysim in ysims if ysim>=ys[method]])/float(n)
    TABLE[method].update({"y":ys[method], "y_max":ymax, "y_ratio":ys[method]/ymax, "y_pval":pvalue})

<a id="mean"></a>
### Predicting biochemical pathway or group membership based on mean vectors
This section looks at how well the biochemical pathways that a particular gene is a member of can be predicted based on the similarity between the vector representation of the phenotype descriptions for that gene and the average vector for all the vector representations of phenotypes asociated with genes that belong to that particular pathway. In calculating the average vector for a given biochemical pathway, the vector corresponding to the gene that is currently being classified is not accounted for, to avoid overestimating the performance by including information about the ground truth during classification. This leads to missing information in the case of biochemical pathways that have only one member. This can be accounted for by only limiting the overall dataset to only include genes that belong to pathways that have atleast two genes mapped to them, and only including those pathways, or by removing the missing values before calculating the performance metrics below.

In [ ]:
# Get the list of methods to look at, and a mapping between each method and the correct similarity metric to apply.
vector_dicts = {k:v.vector_dictionary for k,v in graphs.items()}
methods = list(vector_dicts.keys())
group_id_to_ids = groups.get_group_id_to_ids_dict(dataset.get_gene_dictionary())
valid_group_ids = [group for group,id_list in group_id_to_ids.items() if len(id_list)>1]
valid_ids = [i for i in dataset.get_ids() if len(set(valid_group_ids).intersection(set(id_to_group_ids[i])))>0]
pred_dict = defaultdict(lambda: defaultdict(dict))
true_dict = defaultdict(lambda: defaultdict(dict))
for method in methods:
    for group in valid_group_ids:
        ids = group_id_to_ids[group]
        for identifier in valid_ids:
            # What's the mean vector of this group, without this particular one that we're trying to classify.
            vectors = np.array([vector_dicts[method][some_id] for some_id in ids if not some_id==identifier])
            mean_vector = vectors.mean(axis=0)
            this_vector = vector_dicts[method][identifier]
            pred_dict[method][identifier][group] = 1-metric_dict[method](mean_vector, this_vector)
            true_dict[method][identifier][group] = (identifier in group_id_to_ids[group])*1                

In [ ]:
num_plots, plots_per_row, row_width, row_height = (len(methods), 4, 14, 3)
fig,axs = plt.subplots(math.ceil(num_plots/plots_per_row), plots_per_row, squeeze=False)
for method,ax in zip(methods, axs.flatten()):
    
    # Obtaining the values and metrics.
    y_true = pd.DataFrame(true_dict[method]).as_matrix().flatten()
    y_prob = pd.DataFrame(pred_dict[method]).as_matrix().flatten()
    n_pos, n_neg = Counter(y_true)[1], Counter(y_true)[0]
    precision, recall, thresholds = precision_recall_curve(y_true, y_prob)
    baseline = Counter(y_true)[1]/len(y_true) 
    area = auc(recall, precision)
    auc_to_baseline_auc_ratio = area/baseline
    TABLE[method].update({"mean_auc":area, "mean_baseline":baseline, "mean_ratio":auc_to_baseline_auc_ratio})

    # Producing the precision recall curve.
    step_kwargs = ({'step': 'post'} if 'step' in signature(plt.fill_between).parameters else {})
    ax.step(recall, precision, color='black', alpha=0.2, where='post')
    ax.fill_between(recall, precision, alpha=0.7, color='black', **step_kwargs)
    ax.axhline(baseline, linestyle="--", color="lightgray")
    ax.set_xlabel('Recall')
    ax.set_ylabel('Precision')
    ax.set_ylim([0.0, 1.05])
    ax.set_xlim([0.0, 1.0])
    ax.set_title("PR {0} (Baseline={1:0.3f})".format(method[:10], baseline))
    
fig.set_size_inches(row_width, row_height*math.ceil(num_plots/plots_per_row))
fig.tight_layout()
fig.savefig(os.path.join(OUTPUT_DIR,"prcurve_mean_classifier.png"),dpi=400)
plt.close()

### Predicting biochemical pathway membership based on mean similarity values
This section looks at how well the biochemical pathways that a particular gene is a member of can be predicted based on the average similarity between the vector representationt of the phenotype descriptions for that gene and each of the vector representations for other phenotypes associated with genes that belong to that particular pathway. In calculating the average similarity to other genes from a given biochemical pathway, the gene that is currently being classified is not accounted for, to avoid overestimating the performance by including information about the ground truth during classification. This leads to missing information in the case of biochemical pathways that have only one member. This can be accounted for by only limiting the overall dataset to only include genes that belong to pathways that have atleast two genes mapped to them, and only including those pathways, or by removing the missing values before calculating the performance metrics below.

### Predicting biochemical pathway or group membership with KNN classifier
This section looks at how well the group(s) or biochemical pathway(s) that a particular gene belongs to can be predicted based on a KNN classifier generated using every other gene. For this section, only the groups or pathways which contain more than one gene, and the genes mapped to those groups or pathways, are of interest. This is because for other genes, if we consider them then it will be true that that gene belongs to that group in the target vector, but the KNN classifier could never predict this because when that gene is held out, nothing could provide a vote for that group, because there are zero genes available to be members of the K nearest neighbors.

<a id="output"></a>
### Summarizing the results for this notebook
Write a large table of results to an output file. Columns are generally metrics and rows are generally methods.

In [ ]:
results = pd.DataFrame(TABLE).transpose()
columns = flatten(["Hyperparams","Group","Order","Topic","Data",results.columns])
results["Hyperparams"] = ""
results["Group"] = ""
results["Order"] = np.arange(results.shape[0])
results["Topic"] = TOPIC
results["Data"] = DATA
results = results[columns]
results.reset_index(inplace=True)
results = results.rename({"index":"Method"}, axis="columns")
hyperparam_sep = ":"
results["Hyperparams"] = results["Method"].map(lambda x: x.split(hyperparam_sep)[1] if hyperparam_sep in x else "-")
results["Method"] = results["Method"].map(lambda x: x.split(hyperparam_sep)[0])
results.to_csv(os.path.join(OUTPUT_DIR,"full_table.csv"), index=False)
results.head()